## 0. 구글 드라이브 연결 및 필수 라이브러리 설치 및 로드

### 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### 필수 라이브러리 설치 및 로드

In [ ]:
#!pip install nltk

In [ ]:
#!pip install konlpy

In [ ]:
#from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import nltk
import re

from keras.callbacks import *
import os
import tensorflow as tf

In [ ]:
#nltk.download()

# 이거 안하면 lookuperror 뜬다.(필요한 것만 지정해서 다운로드 해도되고, all을 사용하여 모두 다운로드 해도 된다.)

In [ ]:
# 이후에 데이터 및 파일들을 저장할 경로를 path로 통일하여 따로 저장

path = '/content/gdrive/MyDrive/Colab Notebooks/CUAI 하계프로젝트-NLP/초기 모델링 진행/data/'

***여기 이후부터는 완료된 단계(4. 토큰화, 5. 정수 인코딩 등)로 점프해서 실행시키면 됨!!(노트북을 나누면서 앞부분 삭제함)***

## 6. BERT 모델링

### 6-1. BERT 모델링에 필요한 모듈 및 함수 import  

앞선 단계에서 토큰화, 정수 인코딩, 패딩, 워드 임베딩을 수행했으나   
BERT 모델링에 참고한 책 **"텐서플로 2와 머신러닝으로 시작하는 자연어 처리"**를 보니(p.368~371) 기존에 한 것과 아예 다른 토크나이저를 사용하기 때문에 모델의 입력 데이터를 구성하는 방식이 다르다고 함  
따라서 책에 있는 코드를 따라가며 BERT 모델링에 필요한 모듈 및 함수 import하였음

In [ ]:
!pip install transformers==3.0.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 769 kB 6.9 MB/s 
     |████████████████████████████████| 1.3 MB 60.4 MB/s 
     |████████████████████████████████| 3.0 MB 51.0 MB/s 
     |████████████████████████████████| 880 kB 65.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=ed04480c53cdd9398245014a4c62d141fc916e2e9aaeec894bc289a412952878
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [ ]:
# 모델 학습에 필요한 모듈 import 및 버트의 다국어 토크나이저 다운로드
from transformers import *

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

### 6-2. 버트 문장 전처리 진행하기

버트 모델은 일반적으로 3가지 입력값을 받게 되는데, 요약하면 아래 표와 같음

|입력값|역할|
|-------|--------|
| input_ids | 문장을 토크나이즈해서 인덱스 값으로 변환한다. 일반적으로 버트에서는 단어를 서브워드 단위로 변환시키는 워드피스 토크나이저를 활용한다.  
| attention_mask |어텐션 마스크는 패딩된 부분에 대해 학습에 영향을 받지 않기 위해 처리해주는 입력값이다. 버트 토크나이저에서 1은 어텐션에 영향을 받는 토큰을, 0은 영향을 받지 않는 토큰을 나타낸다.|
|token_type_ids|두 개의 시퀀스를 입력으로 활용할 때 0과 1로 문장의 토큰값을 분리한다.|

버트 토크나이저에서는 문장의 시작이나 끝, 모델별 목적에 맞게 스페셜 토큰을 활용함  
다음은 자주 사용되는 스페셜 토큰들을 요약한 표임

|스페셜 토큰|역할|
|-------|--------|
|[UNK]|모르는 단어에 대한 토큰|
|[MASK]|마스크 토큰, 사전 학습에서 활용|
|[PAD]|최대 길이를 맞추는 용도| 
|[SEP]|문장의 종결을 알림| 
|[CLS]|문장의 시작을 알림|

버트에 필요한 입력값의 형태로 데이터를 변환하는 구조를 직접 구현할 수도 있지만, **허깅페이스의 Tokenizer** 라이브러리를 활용하면 더 쉽고 빠르게 버트의 입력값을 구현할 수 있음  

참고한 책("텐서플로 2와 머신러닝으로 시작하는 자연어 처리")에서는 이 라이브러리 중 버트 토크나이저 작업에 필요한 구조를 만들어주는 **encode_plus** 기능을 활용하였음  
- **encode_plus** 기능: 특정 문장을 버트에 필요한 입력 형태로 변환하는 것뿐만 아니라 문장을 최대 길이에 맞게 패딩까지 해주며, 결과값은 딕셔너리 형태로 출력됨  

변환 순서는 아래 코드와 같음

In [ ]:
def bert_tokenizer(sent, MAX_LEN): 

    encoded_dict = tokenizer.encode_plus(
        text = sent, # 이 부분이 원래 sent1, sent2라고 되어있었는데 뒤에서 not defined 떠서 바꿈
        add_special_tokens = True,    # Add '[CLS]' and '[SEP]'
        max_length = MAX_LEN,         # Pad & truncate all sentences
        pad_to_max_length = True, 
        return_attention_mask = True, # Construct attention masks
        truncation=True
    )

    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask'] # simply differentiates padding from non-padding
    token_type_id = encoded_dict['token_type_ids']  # differentiate two sentences

    return input_id, attention_mask, token_type_id

**encode_plus**의 변환 순서는 다음과 같음
1. 문장을 토크나이징함
2. add_special_tokens를 True로 지정하면 토큰의 시작점에 '[CLS]' 토큰, 토큰의 마지막에 '[SEP]' 토큰을 붙임
3. 각 토큰을 인덱스로 변환함
4. max_length에 MAX_LEN 최대 길이에 따라 문장의 길이를 맞추는 작업을 진행하고, pad_to_max_length 기능을 통해 MAX_LEN의 길이에 미치지 못하는 문장에 패딩을 적용함
5. return_attention_mask 기능을 통해 어텐션 마스크를 생성함
6. 토큰 타입은 문장이 1개일 경우 0으로, 문장이 2개일 경우 0과 1로 구분해서 생성함

In [ ]:
# 간단한 예시로 토크나이저가 잘 동작하는지 확인

# Special Tokens
print(tokenizer.all_special_tokens, '\n', tokenizer.all_special_ids)

['[CLS]', '[MASK]', '[SEP]', '[PAD]', '[UNK]'] 
 [101, 103, 102, 0, 100]


In [ ]:
# Test Tokenizers
kor_encode = tokenizer.encode("안녕하세요, 반갑습니다")
eng_encode = tokenizer.encode("Hello world")
kor_decode = tokenizer.decode(kor_encode)
eng_decode = tokenizer.decode(eng_encode)

print(kor_encode)
print(eng_encode)
print(kor_decode)
print(eng_decode)

[101, 9521, 118741, 35506, 24982, 48549, 117, 9321, 118610, 119081, 48345, 102]
[101, 31178, 11356, 102]
[CLS] 안녕하세요, 반갑습니다 [SEP]
[CLS] Hello world [SEP]


### 6-3. final_data를 train/test로 나눈 후 train 데이터 전처리

In [ ]:
# 후에 나올 주요 변수들 정의
MAX_LEN = 25 # okt 형태소 분석기로 분석했을 때 95%의 데이터들이 살아남은 지점의 길이
NUM_EPOCHS = 3 # 임의로 설정
BATCH_SIZE = 64 # 임의로 설정
VALID_SPLIT = 0.1 # 임의로 설정

In [ ]:
# final_data 로드
# (참고)path 부분은 맨 앞 0장의 마지막 코드에서 경로 지정하였음!

data = pd.read_excel(path + 'final_data.xlsx', engine='openpyxl')

In [ ]:
data.tail(3)

,Unnamed: 0,document,label
303472,303472,당장 내일 밥은 어떡하고 내 병원비는 어쩌지,0
303473,303473,응 이제 다들 독립해가지고 명절 아니면 찾아오질 않아,0
303474,303474,안부전화라도 해주면 좋을 텐데,0


In [ ]:
# 책에 train과 test 데이터를 나눴으므로 train_test_split을 이용해 final_data를 분리
# test_size = 0.25가 defalut, 랜덤 시드 42로 고정
X_train, X_test, y_train, y_test = train_test_split(data['document'],data['label'], 
                                                    test_size = 0.25, stratify = data['label'], random_state=42)

In [ ]:
# 다시 train과 test끼리 묶어주고 이것을 일단 txt 파일로 저장
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

train_data.to_csv(path + 'final_train_data.txt', sep = '\t', index = False)
test_data.to_csv(path + 'final_test_data.txt', sep = '\t', index = False)

6-2에서 만든 토크나이저와 encode_plus 기능을 활용해 데이터를 전처리

아래 코드 중 train_data_sent_pads 부분 짤 때 참고한 사이트: https://github.com/changwookjun/lgsp_nlp_practice/blob/master/Bert_Naver_movie_tf_mentee.ipynb

In [ ]:
input_ids = []
attention_masks = []
token_type_ids = []
train_data_labels = []

def clean_text(sent):
    sent_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", " ", sent)
    return sent_clean

for train_sent, train_label in train_data[['document', 'label']].values: 
    try:
        # 아래 코드에 MAX_LEN에도 숫자를 넣어줘야 할 것 같음
        # 여기서는 버트 토크나이저의 word piece model을 통해 파악한 토큰 길이 분포 중
        # 제3사분위 값인 39를 정했다고 함
        input_id, attention_mask, token_type_id = bert_tokenizer(clean_text(train_sent), MAX_LEN)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        train_data_labels.append(train_label)
    
    except Exception as e:
        print(e)
        print(train_sent)
        pass


train_input_ids = np.array(input_ids, dtype=int)
train_attention_masks = np.array(attention_masks, dtype=int)
train_type_ids = np.array(token_type_ids, dtype=int)
train_inputs = (train_input_ids, train_attention_masks, train_type_ids)

# 책에서는 이 변수 정의한 부분이 없었는데 정작 학습에서는 등장해서 구글링으로 대충 찾아봄
train_data_sent_pads = pad_sequences(train_input_ids, maxlen=MAX_LEN, padding='pre')
train_data_labels = np.asarray(train_data_labels, dtype=np.int32) # 정답 토크나이징 리스트

print("# sents: {}, # labels: {}".format(len(train_input_ids), len(train_data_labels)))

# sents: 227606, # labels: 227606


위 전처리 과정 설명  
- 우선 데이터에서 한글 외 텍스트를 제거하기 위한 clean_text 함수를 적용
- 버트 토크나이저를 활용한 인코딩 진행
- 최종 출력값은 넘파이로 변환한 후 train_inputs에 튜플 형태로 묶어서 저장하는 형태로 구성

In [ ]:
# 원하면 책 p.373에 있는 코드로 예시 확인해 볼 수 있게 남겨둔 여백

### 6-4. final_train_data를 모델에 학습시키기

In [ ]:
# 학습을 준비하기 위해 버트 분류 클래스를 구성하고, 최적화, 손실값, 및 정확도를 선언하고 학습을 진행하기 위한 compile을 진행
class TFBertClassifier(tf.keras.Model):
    def __init__(self, model_name, dir_path, num_class):
        super(TFBertClassifier, self).__init__()

        self.bert = TFBertModel.from_pretrained(model_name, cache_dir=dir_path)
        self.dropout = tf.keras.layers.Dropout(self.bert.config.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(
            num_class,
            kernel_initializer = tf.keras.initializers.TruncatedNormal(
                self.bert.config.initializer_range
            ),
            name="classifier"
        )
    
    def call(self, inputs, attention_mask=None, token_type_ids=None, training=False):

        # outputs 값: # sequence_output, pooled_output, (hidden_states), (attentions)
        outputs = self.bert(
            inputs, attention_mask=attention_mask, token_type_ids=token_type_ids
        )
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output, training=training)
        logits = self.classifier(pooled_output)

        return logits

cls_model = TFBertClassifier(model_name='bert-base-multilingual-cased', 
                             dir_path='bert_ckpt', 
                             num_class=2)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


TFBertClassifier 클래스 설명  
: 이 클래스는 사전 학습된 버트 모델을 불러와 그 위에 완전연결층 1층을 쌓는 구조로 되어 있음  

\__init__ 부분
- model_name, dir_path, num_class를 인자로 받아 활용할 모델 이름, 모델이 저장된 위치, 클래스의 수를 모델에 지정함
- self.bert 부분의 TFBertModel.from_pretrained를 통해 기존에 사전 학습했던 부분이 로드됨
- TFBertModel의 결과값은 sequence_output, pooled_output, hidden_states, attentions로 구성됨
- num_class에 원하는 분류 값을 추가해서 정답의 개수를 정할 수 있음, 감정 분석 데이터에서는 정답이 2개(긍정, 부정)으로 나뉨  

call 메소드 부분  
- inputs를 통해 문장 텍스트를 받은 다음,
- 버트에서 outputs로 결과값을 추출한 다음,
- self.classifier를 통해 완전 연결층을 활용해 최종적으로 self.num_labels 개수에 맞는 예측값을 출력하는 구조  

이 구조에서 핵심은 사전 학습된 부분을 불러와서 그 이후에 다양한 종류의 자연어 문제에 적용하는 것임

In [ ]:
# 앞에서 구성한 TFBertClassifier를 활용해 최적화, 손실값 및 평가 기준을 설정하는 과정
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

cls_model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=[metric])

- 최적화: 아담 최적화 사용
- 손실값: SparseCategoricalCrossentropy 사용
- 평가 기준: 모델의 정확도를 측정하는 SparseCategoricalAccuracy를 정의  

위 기준을 모델의 compile 함수에 넣어 모델을 학습할 준비를 모두 마침

학습시키기 전에 GPU 활용을 위한 코드를 삽입해보았음  
참고 사이트: https://m.blog.naver.com/demian7607/222043724449 

In [ ]:
# 1) GPU 디바이스 확인
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Found GPU at: /device:GPU:0


In [ ]:
# 이제 학습을 시작
# 워낙 큰 파라미터를 사용하기 때문에 CPU에서의 학습은 거의 불가능
# 따라서 웬만하면 반드시 GPU를 활용해야 함
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=2)

# 이부분 경로는 일단 그냥 구글드라이브의 data 모아놨던 곳으로 코드 바꿈
# model_name 자리에도 위에 적혀있던 'bert-base-multilingual-cased' 적어놓음
checkpoint_path = os.path.join(path, 'bert-base-multilingual-cased', 'weights.h5')
checkpoint_dir = os.path.dirname(checkpoint_path)

if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else: 
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))

cp_callback = ModelCheckpoint(
    checkpoint_path, 
    monitor="val_accuracy", 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=True, 
)


with tf.device('/device:GPU:0'):
    history = cls_model.fit(
        train_data_sent_pads, # 책에는 train_data_sent_pads로 되어있는데 어디에도 이렇게 정의한 변수가 없음
        train_data_labels, 
        epochs=NUM_EPOCHS, # 앞에 6-3에서 임의로 정함
        batch_size=BATCH_SIZE, # 앞에 6-3에서 임의로 정함
        validation_split=VALID_SPLIT, # 앞에 6-3에서 임의로 정함
        callbacks=[earlystop_callback, cp_callback]
    )

/content/gdrive/MyDrive/Colab Notebooks/CUAI 하계프로젝트-NLP/초기 모델링 진행/data/bert-base-multilingual-cased -- Folder already exists 

Epoch 1/3
3201/3201 [==============================] - ETA: 0s - loss: 0.6659 - accuracy: 0.6230
Epoch 1: val_accuracy improved from -inf to 0.62550, saving model to /content/gdrive/MyDrive/Colab Notebooks/CUAI 하계프로젝트-NLP/초기 모델링 진행/data/bert-base-multilingual-cased/weights.h5
3201/3201 [==============================] - 1221s 381ms/step - loss: 0.6659 - accuracy: 0.6230 - val_loss: 0.6623 - val_accuracy: 0.6255
Epoch 2/3
3201/3201 [==============================] - ETA: 0s - loss: 0.6686 - accuracy: 0.6196
Epoch 2: val_accuracy did not improve from 0.62550
3201/3201 [==============================] - 1213s 379ms/step - loss: 0.6686 - accuracy: 0.6196 - val_loss: 0.6628 - val_accuracy: 0.6255
Epoch 3/3
3201/3201 [==============================] - ETA: 0s - loss: 0.6628 - accuracy: 0.6256
Epoch 3: val_accuracy did not improve f

### 6-4. final_test_data로 모델의 성능 테스트하기

In [ ]:
# 이제 test 데이터로 테스트를 진행
input_ids = []
attention_masks = []
token_type_ids = []
test_data_labels = []

for test_sent, test_label in zip(test_data['document'], test_data['label']): 
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(clean_text(test_sent), MAX_LEN)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        test_data_labels.append(test_label)
    
    except Exception as e:
        print(e)
        print(test_sent)
        pass

test_input_ids = np.array(input_ids, dtype=int)
test_attention_masks = np.array(attention_masks, dtype=int)
test_type_ids = np.array(token_type_ids, dtype=int)
test_inputs = (test_input_ids, test_attention_masks, test_type_ids)

test_data_labels = np.asarray(test_data_labels, dtype=np.int32) # 정답 토크나이징 리스트

print(
    "num sents, labels {}, {}".format(len(test_input_ids), len(test_data_labels))
)

results = cls_model.evaluate(test_inputs, test_data_labels, batch_size=512)
print("test loss, test acc:", results)

num sents, labels 75869, 75869
149/149 [==============================] - 117s 767ms/step - loss: 0.6613 - accuracy: 0.6256
test loss, test acc: [0.6612667441368103, 0.6256178617477417]


test_data에 모델을 테스트해본 결과 accuracy가 62.56%로 굉장히 나쁜 성능을 보였음  
그전에 썼던 GPT2나 다른 KoBERT, Bi-LSTM 모델의 결과와 비교해 본 뒤 가장 성능이 좋은 모델을 선택하면 될 것 같음

# END